In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def block(input_layer,
          channel_size,
          num_block,
          is_50 = False,
          first_block = False
          ):

    x = input_layer

    prev_layer_50 = x

    #블록 수만큼 회전
    for i in range(num_block):

        #첫 번째 블록의 첫 번째 CNN에서만 strides를 적용하기 위함.
        if first_block and i == 0:
            strides = (2,2)
        else:
            strides = (1,1)

        # 50일 경우 1*1 레이어 활성화
        if is_50 :
            x = keras.layers.Conv2D(filters = channel_size, kernel_size = (1,1), padding = 'same', strides=strides)(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

        # 50일 경우 3*3 레이어 1회
            x = keras.layers.Conv2D(filters = channel_size, kernel_size = (3,3), padding = 'same')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

        # 50일 경우 1*1 레이어 1회
            x = keras.layers.Conv2D(filters = channel_size*4, kernel_size = (1,1), padding = 'same')(x)
            x = keras.layers.BatchNormalization()(x)
            if num_block-1 == i:
                #prev_layer_50의 채널 수를 맞춰 주기 위해 필터를 조절합니다.
                prev_layer_50 = keras.layers.Conv2D(filters=channel_size*4, kernel_size=(1,1), padding='same', strides=strides)(prev_layer_50)
                x = keras.layers.Add()([x, prev_layer_50])
                x = keras.layers.Activation('relu')(x)
            else:
                x = keras.layers.Activation('relu')(x)

        # 50이 아닐 경우 3*3 레이어 2회
        else :
            prev_layer_not50 = x

            x = keras.layers.Conv2D(filters = channel_size, kernel_size = (3,3), padding = 'same', strides=strides)(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)
            x = keras.layers.Conv2D(filters = channel_size, kernel_size = (3,3), padding = 'same')(x)
            x = keras.layers.BatchNormalization()(x)

            if num_block-1 == i:
                prev_layer_not50 = keras.layers.Conv2D(filters = channel_size, kernel_size = (1,1), padding = 'same')(prev_layer_not50)
                x = keras.layers.Add()([x, prev_layer_not50])
                x = keras.layers.Activation('relu')(x)

    return x

In [3]:
def build_resnet(input_shape,
                 num_channel_list = [64, 128, 256, 512],
                 num_block_list = [3, 4, 6, 3],
                 is_50 = False,
                 num_classes=1
                 ):

    input_layer = keras.layers.Input(input_shape)
    output = input_layer

    output = keras.layers.Conv2D(filters = 64, kernel_size = (7,7), padding = 'same', name = "conv1", strides = (2,2))(output)
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2))(output)

    for i, (num_cnn, num_block) in enumerate(zip(num_channel_list, num_block_list)):
        first_block = (i==0)
        output = block(output, channel_size = num_cnn, num_block = num_block, is_50 = is_50, first_block=first_block)

    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(1000, activation='relu', name='fc1')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)

    model = keras.Model(
        inputs=input_layer,
        outputs=output
    )
    return model

In [4]:
resnet_34 = build_resnet(input_shape=(32,32,3), is_50 = False)

resnet_34.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 16, 16, 64)           9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 16, 16, 64)           256       ['conv1[0][0]']               
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 16, 16, 64)           0         ['batch_normalization[0][0

PlainNet 제작

In [ ]:
def plain_block(input_layer,
          channel_size,
          num_block,
          is_50 = False
          ):

    x = input_layer

    #블록 수만큼 회전
    for i in range(num_block):

        # 50일 경우 1*1 레이어 활성화
        if is_50 :
            x = keras.layers.Conv2D(filters = channel_size, kernel_size = (1,1), padding = 'same')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

        # 50일 경우 3*3 레이어 1회
            x = keras.layers.Conv2D(filters = channel_size, kernel_size = (3,3), padding = 'same')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

        # 50일 경우 1*1 레이어 1회
            x = keras.layers.Conv2D(filters = channel_size*4, kernel_size = (1,1), padding = 'same')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

        # 50이 아닐 경우 3*3 레이어 2회
        else :
            for j in range(2):
                x = keras.layers.Conv2D(filters = channel_size, kernel_size = (3,3), padding = 'same')(x)
                x = keras.layers.BatchNormalization()(x)
                x = keras.layers.Activation('relu')(x)

    return x

In [ ]:
def build_plainnet(input_shape,
                 num_channel_list = [64, 128, 256, 512],
                 num_block_list = [3, 4, 6, 3],
                 is_50 = False,
                 num_classes=1
                 ):

    input_layer = keras.layers.Input(input_shape)
    output = input_layer

    output = keras.layers.Conv2D(filters = 64, kernel_size = (7,7), padding = 'same', name = "conv1", strides = (2,2))(output)
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2))(output)

    for i, (num_cnn, num_block) in enumerate(zip(num_channel_list, num_block_list)):
        output = plain_block(output, channel_size = num_cnn, num_block = num_block, is_50 = is_50)
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(1000, activation='relu', name='fc1')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)

    model = keras.Model(
        inputs=input_layer,
        outputs=output
    )
    return model